# Scratch Artificial Neural Network

In [1]:
import sys

sys.dont_write_bytecode = True

# Import libraries
# External libraries
import torch


# Import Libraries
import numpy as np
import pandas as pd
from ArtificialNeuralNetwork import ArtificialNeuralNetwork
from Function import ActivationFunction, LossFunction
from enums import InitializerType

In [2]:
# Disable __grad based on assistant direction
torch.set_grad_enabled(False)

# Pipeline for testing MNIST dataset

In [3]:
# Const variables
input_size = 784
hidden_layers = 1
output_size = 10
learning_rate = 0.01
param_1 = 0
param_2 = 0.5

In [4]:
# Import Dataset
train = pd.read_csv("data/train.csv")

In [5]:
# Data Preprocessing
data = np.array(train)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape
X_train = X_train.T

In [6]:
# Initialize Artificial Neural Network
ann = ArtificialNeuralNetwork(input_size, output_size, hidden_layers, 128, InitializerType.RANDOM_DIST_NORMAL, InitializerType.RANDOM_DIST_NORMAL, param_1, param_2, 0, ActivationFunction.sigmoid, LossFunction.mean_squared_error)

In [ ]:
# Train Artificial Neural Network
ann.train(torch.tensor(X_train), torch.tensor(Y_train), learning_rate, 2, LossFunction.mean_squared_error, verbose = True)

Iter 1000 - Loss: 30.39834976196289
